In [2]:
import io
import os
import re
import keras
import tensorflow as tf
import numpy as np # used for handling numbers
import pandas as pd # used for handling the dataset
df= pd.read_csv('drive/My Drive/Colab Notebooks/dataset/labeled_outs.txt',delimiter=",",header=None)
#kw=pd.read_csv('drive/My Drive/Colab Notebooks/dataset/temp_frequent.txt',header=None)
# Dataset is now stored in a Pandas Dataframe

Using TensorFlow backend.


In [0]:
df.columns=['label','text']
df.text=df.text.astype(str)
df.label=df.label.astype(str)
df=df.sample(frac=1)
docs=df['text'].values
labels=df['label'].values

In [0]:

from sklearn.feature_extraction.text import CountVectorizer
input_dim=5000
count_vect = CountVectorizer(max_features = input_dim)
X_train_counts = count_vect.fit_transform(docs)

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape
X_train_tfidf = X_train_tfidf.toarray()
X_train_tfidf = np.reshape(X_train_tfidf, (X_train_tfidf.shape[0], 1, -1))
print(X_train_tfidf.shape)

(91795, 1, 5000)


In [0]:

kw.columns=['words']
dfreq={}
for k in kw['words']:
  dfreq[k]=0
print(len(dfreq))

In [0]:
for d in docs:
  words=re.split('[\s]',d)
  words=np.unique(words)
  for w in words:
    if w in dfreq:
      dfreq[w]=dfreq[w]+1

tf_idf = []
for d in docs:
  words=re.split('[\s]',d)
  N=len(words)
  tfreq={}
  for w in words:
    if w in tfreq:
      tfreq[w]=tfreq[w]+1
    else:
      tfreq[w]=1
  temp=[]
  for w in dfreq:
    if w in tfreq:
      tf=tfreq[w]*1.0/N
      idf=np.log(len(docs)*1.0/dfreq[w])
      temp.append(tf*idf)
    else:
      temp.append(0.0)
  tf_idf.append(temp)

tf_idf = np.array(tf_idf)
input_dim=len(tf_idf[0])
print(input_dim)
tf_idf = np.reshape(tf_idf, (tf_idf.shape[0], 1, tf_idf.shape[1]))

19999


In [6]:

from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
print(labels)
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
print(dummy_y)


['sports' 'state' 'entertainment' ... 'international' 'economy'
 'arts_literature']
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, dummy_y, test_size=0.18)

In [8]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(75271, 1, 5000)
(75271, 8)
(16524, 1, 5000)
(16524, 8)


In [0]:
from keras.models import Sequential
from keras.layers import GRU, Dense, Dropout

model = Sequential()
model.add(GRU(2048, input_shape=(X_train.shape[1],X_train.shape[2]),return_sequences=True))
model.add(GRU(1024,return_sequences=True,dropout=0.5,recurrent_dropout=0.3))
model.add(GRU(512,return_sequences=True,dropout=0.5,recurrent_dropout=0.3))
model.add(GRU(128,return_sequences=True,dropout=0.5,recurrent_dropout=0.3))
model.add(GRU(64,return_sequences=True))
model.add(GRU(32))
model.add(Dense(8, activation='softmax'))

In [10]:

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['acc'])
# summarize the model
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 1, 2048)           43309056  
_________________________________________________________________
gru_2 (GRU)                  (None, 1, 1024)           9440256   
_________________________________________________________________
gru_3 (GRU)                  (None, 1, 512)            2360832   
_________________________________________________________________
gru_4 (GRU)                  (None, 1, 128)            246144    
_________________________________________________________________
gru_5 (GRU)                  (None, 1, 64)             37056     
_________________________________________________________________
gru_6 (GRU)                  (None, 32)                9312      
_________________________________________________________________
dense_1 (Dense)              (None, 8)                

In [12]:
history = model.fit(X_train, y_train, epochs=25,batch_size=256,shuffle=False, verbose=1,validation_split=0.1)
#model.save_weights('./checkpoints/text_classifier_rnn_tfidf')
!mkdir -p saved_model
model.save('saved_model/text_classifier_rnn_tfidf')

Train on 67743 samples, validate on 7528 samples
Epoch 1/25
67743/67743 [==============================] - 35s 516us/step - loss: 0.7161 - acc: 0.7637 - val_loss: 0.3671 - val_acc: 0.8912
Epoch 2/25
67743/67743 [==============================] - 31s 454us/step - loss: 0.3138 - acc: 0.9071 - val_loss: 0.3325 - val_acc: 0.9038
Epoch 3/25
67743/67743 [==============================] - 31s 455us/step - loss: 0.2486 - acc: 0.9267 - val_loss: 0.3323 - val_acc: 0.9036
Epoch 4/25
67743/67743 [==============================] - 31s 458us/step - loss: 0.2113 - acc: 0.9373 - val_loss: 0.3458 - val_acc: 0.9056
Epoch 5/25
67743/67743 [==============================] - 31s 456us/step - loss: 0.1845 - acc: 0.9452 - val_loss: 0.3468 - val_acc: 0.9058
Epoch 6/25
67743/67743 [==============================] - 31s 455us/step - loss: 0.1635 - acc: 0.9517 - val_loss: 0.3650 - val_acc: 0.9053
Epoch 7/25
67743/67743 [==============================] - 31s 455us/step - loss: 0.1514 - acc: 0.9545 - val_loss: 0.3

In [0]:
#model.load_weights('./checkpoints/text_classifier_rnn_tfidf')
new_model = tf.keras.models.load_model('saved_model/text_classifier_rnn_tfidf')

In [13]:
results = model.evaluate(X_test, y_test)
print(results)

16524/16524 [==============================] - 6s 358us/step
[0.522150849361226, 0.8983902335166931]


In [0]:
y_pred=model.predict(X_test)

In [15]:
"""
import numpy as np
array1=[[ 0,  1,  2,  3,  4],
       [ 0,  1,  2,  3,  4],
       [5, 6, 7, 1, 0],
       [5, 6, 7, 1, 0]]
array2=[[ 5, 6, 7, 1, 0],
       [ 0,  1,  2,  3,  4],
       [0,  1,  2,  3,  4],
       [5, 6, 7, 1, 0]]
"""
no_cat=len(y_test[0])
conf_mat=np.zeros(no_cat*no_cat).reshape(no_cat,no_cat)
test_size=len(y_test)
print(conf_mat.shape)
for i in range(test_size):
  y_t=y_test[i]
  y_p=y_pred[i]
  class_t=np.where(y_t == np.amax(y_t))
  class_t=class_t[0][0]
  class_p=np.where(y_p == np.amax(y_p))
  class_p=class_p[0][0]
  conf_mat[class_t][class_p]=conf_mat[class_t][class_p]+1

(8, 8)


In [16]:
tp=np.zeros(no_cat)
fp=np.zeros(no_cat)
tn=np.zeros(no_cat)
fn=np.zeros(no_cat)

prec=np.zeros(no_cat)
rec=np.zeros(no_cat)
f_score=np.zeros(no_cat)

for i in range(no_cat):
  for j in range(no_cat):
    if i==j:
      tp[i]=conf_mat[i][j]
    if i!= j:
      fp[i] = fp[i]+conf_mat[j][i]
      fn[i] = fn[i]+conf_mat[i][j]
    for k in range(no_cat):
      if i!=k & j!=k:
        tn[i]=tn[i]+conf_mat[j][k]

for i in range(no_cat):
    prec[i] = tp[i] * 100.0 / (tp[i] + fp[i])
    rec[i] = tp[i] * 100.0 / (tp[i] + fn[i])
    f_score[i] = 2 * prec[i] * rec[i] / (prec[i] + rec[i])

print(prec)
print(rec)
print(f_score)

[93.68421053 89.29551693 89.40206186 94.12559082 83.2807571  89.18469218
 93.97993311 87.42378049]
[92.57057949 87.65154917 90.18302829 88.45177665 89.66521106 86.24296058
 95.98633646 86.50075415]
[93.12406577 88.46589622 89.79084697 91.20052339 86.35514019 87.68916155
 94.97253908 86.95981804]
